In [1]:
import requests
from datetime import timedelta, datetime

import pandas as pd
from flask import jsonify
from prefect import task, Flow
from prefect.schedules import IntervalSchedule

import joblib
# MODEL = joblib.load('best_gs_pipeline.pkl')

@task
def predict(input_data_path:str,output_data_path:str):
    """
    This task load the saved model, input data and returns prediction.
    If failed this task will retry 3 times at 5 min interval and fail permenantly.
    """
    MODEL = joblib.load('best_gs_pipeline.pkl')
    df = pd.read_csv(input_data_path)
    data = df.iloc[:,:-1]
    predicted = MODEL.predict(data)
    np.savetxt(output_data_path, predicted)
    return "OK"

    
schedule = IntervalSchedule(interval=timedelta(minutes=1))

# Attach the schedule object and orchastrate the workflow.
with Flow("predictions", schedule=schedule) as flow:
    predicted = predict("test.csv","output_data.csv")
flow.run()

[2022-03-21 21:06:32-0300] INFO - prefect.predictions | Waiting for next scheduled run at 2022-03-22T00:07:00+00:00
[2022-03-21 21:07:00-0300] INFO - prefect.FlowRunner | Beginning Flow run for 'predictions'
[2022-03-21 21:07:00-0300] INFO - prefect.TaskRunner | Task 'predict': Starting task run...
[2022-03-21 21:07:01-0300] INFO - prefect.TaskRunner | Task 'predict': Finished task run for task with final state: 'Success'
[2022-03-21 21:07:01-0300] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2022-03-21 21:07:01-0300] INFO - prefect.predictions | Waiting for next scheduled run at 2022-03-22T00:08:00+00:00


KeyboardInterrupt: 